# Starting to explore daily train data

In [6]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import time
from dateutil.relativedelta import relativedelta

# Hack for using import with relative path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fetchTrainData import fetch_train_data
from fetchWeatherData import fetch_weather_data, save_weather_data, open_weather_data

weather_data = {}

def get_weather(stop_):
    tag = stop_['station.shortCode']
    if type(tag) == str:
        lat = stop_['station.location'][0]
        lon = stop_['station.location'][1]
        if tag not in weather_data:
            print("FETCHING DATA FOR: " + tag)
            weather_data[tag] = fetch_weather_data(unix_time, lat, lon)['list'][0]['main']['temp']

            
def extract_time_tables(train_stops_):
    print("TRAIN NUMBER " + str(train_stops_.name) + ", " + str(float(train_stops_.name) / number_of_trains))
    train_stops = pd.json_normalize(train_stops_)
    to_return = train_stops['differenceInMinutes'].sum()
    train_stops = train_stops.drop(['differenceInMinutes'], axis=1)

    train_stops.apply(get_weather, axis=1)

    for stop in train_stops['station.shortCode'].unique():
        if type(stop) == str:
            if stop not in df.columns:
                df[stop] = 0.0

            df[stop][train_stops_.name] = weather_data[stop]
        
    
    return to_return


loops = 12 # How many months data we want in the dataframe
number_of_trains = 100 # How many trains to fetch per month. Seems to be softlocked at about 1000, 500 is safe
all_data_frames = []
all_late_amounts = []


for i in range(loops):
    date = datetime.date.today() - relativedelta(months = i)
    unix_time = time.mktime(date.timetuple())
    print("LOOP " + str(i) + ", DATE: " + str(date))

    q = (
        """ 
        {
        """
          f'  trainsByDepartureDate(departureDate: \"{date}\",'
        """
        """
          f'  take: {number_of_trains},'
        """
            where: {
                and: [
                    {or: [{deleted: {unequals: null}}, 
                    {deleted: {equals: false}}]}, 
                    {cancelled: {equals: false}}, 
                    {operator: {shortCode: {equals: "vr"}}}
                    ]  
            }
          ) 
          {
            trainNumber
            departureDate
            timeTableRows {
            differenceInMinutes
              station {
                shortCode
                location
              }
            }
          }
        }
        """
        )

    result = fetch_train_data(q)
    recs = result['data']['trainsByDepartureDate']

    df = pd.json_normalize(recs)
    all_stops = pd.json_normalize(df['timeTableRows'])
    df = df.drop(['timeTableRows'], axis=1)

    weather_data = open_weather_data(date)
    how_much_late = all_stops.apply(extract_time_tables, axis=1)
    save_weather_data(date, weather_data)
    
    df['departureDate'] = i
    
    all_data_frames.append(df)
    all_late_amounts.append(how_much_late)
    
combined_train_data = pd.concat(all_data_frames).fillna(0).reset_index().drop(['index'], axis=1)
combined_late_amounts = pd.concat(all_late_amounts).fillna(0).reset_index().drop(['index'], axis=1)

print(combined_train_data)
print(combined_late_amounts)

LOOP 0, DATE: 2021-10-26
TRAIN NUMBER 0, 0.0
TRAIN NUMBER 1, 0.01
TRAIN NUMBER 2, 0.02
TRAIN NUMBER 3, 0.03
TRAIN NUMBER 4, 0.04
TRAIN NUMBER 5, 0.05
TRAIN NUMBER 6, 0.06
TRAIN NUMBER 7, 0.07
TRAIN NUMBER 8, 0.08
TRAIN NUMBER 9, 0.09
TRAIN NUMBER 10, 0.1
TRAIN NUMBER 11, 0.11
TRAIN NUMBER 12, 0.12
TRAIN NUMBER 13, 0.13
TRAIN NUMBER 14, 0.14
TRAIN NUMBER 15, 0.15


/tmp/ipykernel_24800/3857213270.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[stop] = 0.0


TRAIN NUMBER 16, 0.16
TRAIN NUMBER 17, 0.17
TRAIN NUMBER 18, 0.18
TRAIN NUMBER 19, 0.19
TRAIN NUMBER 20, 0.2
TRAIN NUMBER 21, 0.21
TRAIN NUMBER 22, 0.22
TRAIN NUMBER 23, 0.23
TRAIN NUMBER 24, 0.24
TRAIN NUMBER 25, 0.25
TRAIN NUMBER 26, 0.26
TRAIN NUMBER 27, 0.27
TRAIN NUMBER 28, 0.28
TRAIN NUMBER 29, 0.29
TRAIN NUMBER 30, 0.3
TRAIN NUMBER 31, 0.31
TRAIN NUMBER 32, 0.32
TRAIN NUMBER 33, 0.33
TRAIN NUMBER 34, 0.34
TRAIN NUMBER 35, 0.35
TRAIN NUMBER 36, 0.36
TRAIN NUMBER 37, 0.37
TRAIN NUMBER 38, 0.38
TRAIN NUMBER 39, 0.39
TRAIN NUMBER 40, 0.4
TRAIN NUMBER 41, 0.41
TRAIN NUMBER 42, 0.42
TRAIN NUMBER 43, 0.43
TRAIN NUMBER 44, 0.44
TRAIN NUMBER 45, 0.45
TRAIN NUMBER 46, 0.46
TRAIN NUMBER 47, 0.47
TRAIN NUMBER 48, 0.48
TRAIN NUMBER 49, 0.49
TRAIN NUMBER 50, 0.5
TRAIN NUMBER 51, 0.51
TRAIN NUMBER 52, 0.52
TRAIN NUMBER 53, 0.53
TRAIN NUMBER 54, 0.54
TRAIN NUMBER 55, 0.55
TRAIN NUMBER 56, 0.56
TRAIN NUMBER 57, 0.57
TRAIN NUMBER 58, 0.58
TRAIN NUMBER 59, 0.59
TRAIN NUMBER 60, 0.6
TRAIN NUMBER 61

LOOP 4, DATE: 2021-06-26
TRAIN NUMBER 0, 0.0
TRAIN NUMBER 1, 0.01
TRAIN NUMBER 2, 0.02
TRAIN NUMBER 3, 0.03
TRAIN NUMBER 4, 0.04
TRAIN NUMBER 5, 0.05
TRAIN NUMBER 6, 0.06
TRAIN NUMBER 7, 0.07
TRAIN NUMBER 8, 0.08
TRAIN NUMBER 9, 0.09
TRAIN NUMBER 10, 0.1
TRAIN NUMBER 11, 0.11
TRAIN NUMBER 12, 0.12
TRAIN NUMBER 13, 0.13
TRAIN NUMBER 14, 0.14
TRAIN NUMBER 15, 0.15
TRAIN NUMBER 16, 0.16
TRAIN NUMBER 17, 0.17
TRAIN NUMBER 18, 0.18
TRAIN NUMBER 19, 0.19
TRAIN NUMBER 20, 0.2
TRAIN NUMBER 21, 0.21
TRAIN NUMBER 22, 0.22
TRAIN NUMBER 23, 0.23
TRAIN NUMBER 24, 0.24
TRAIN NUMBER 25, 0.25
TRAIN NUMBER 26, 0.26
TRAIN NUMBER 27, 0.27
TRAIN NUMBER 28, 0.28
TRAIN NUMBER 29, 0.29
TRAIN NUMBER 30, 0.3
TRAIN NUMBER 31, 0.31
TRAIN NUMBER 32, 0.32
TRAIN NUMBER 33, 0.33
TRAIN NUMBER 34, 0.34
TRAIN NUMBER 35, 0.35
TRAIN NUMBER 36, 0.36
TRAIN NUMBER 37, 0.37
TRAIN NUMBER 38, 0.38
TRAIN NUMBER 39, 0.39
TRAIN NUMBER 40, 0.4
TRAIN NUMBER 41, 0.41
TRAIN NUMBER 42, 0.42
TRAIN NUMBER 43, 0.43
TRAIN NUMBER 44, 0.44


TRAIN NUMBER 76, 0.76
TRAIN NUMBER 77, 0.77
TRAIN NUMBER 78, 0.78
TRAIN NUMBER 79, 0.79
TRAIN NUMBER 80, 0.8
TRAIN NUMBER 81, 0.81
TRAIN NUMBER 82, 0.82
TRAIN NUMBER 83, 0.83
TRAIN NUMBER 84, 0.84
TRAIN NUMBER 85, 0.85
TRAIN NUMBER 86, 0.86
TRAIN NUMBER 87, 0.87
TRAIN NUMBER 88, 0.88
TRAIN NUMBER 89, 0.89
TRAIN NUMBER 90, 0.9
TRAIN NUMBER 91, 0.91
TRAIN NUMBER 92, 0.92
TRAIN NUMBER 93, 0.93
TRAIN NUMBER 94, 0.94
TRAIN NUMBER 95, 0.95
TRAIN NUMBER 96, 0.96
TRAIN NUMBER 97, 0.97
TRAIN NUMBER 98, 0.98
TRAIN NUMBER 99, 0.99
LOOP 8, DATE: 2021-02-26
TRAIN NUMBER 0, 0.0
TRAIN NUMBER 1, 0.01
TRAIN NUMBER 2, 0.02
TRAIN NUMBER 3, 0.03
TRAIN NUMBER 4, 0.04
TRAIN NUMBER 5, 0.05
TRAIN NUMBER 6, 0.06
TRAIN NUMBER 7, 0.07
TRAIN NUMBER 8, 0.08
TRAIN NUMBER 9, 0.09
TRAIN NUMBER 10, 0.1
TRAIN NUMBER 11, 0.11
TRAIN NUMBER 12, 0.12
TRAIN NUMBER 13, 0.13
TRAIN NUMBER 14, 0.14
TRAIN NUMBER 15, 0.15
TRAIN NUMBER 16, 0.16
TRAIN NUMBER 17, 0.17
TRAIN NUMBER 18, 0.18
TRAIN NUMBER 19, 0.19
TRAIN NUMBER 20, 0.2


TRAIN NUMBER 51, 0.51
TRAIN NUMBER 52, 0.52
TRAIN NUMBER 53, 0.53
TRAIN NUMBER 54, 0.54
TRAIN NUMBER 55, 0.55
TRAIN NUMBER 56, 0.56
TRAIN NUMBER 57, 0.57
TRAIN NUMBER 58, 0.58
TRAIN NUMBER 59, 0.59
TRAIN NUMBER 60, 0.6
TRAIN NUMBER 61, 0.61
TRAIN NUMBER 62, 0.62
TRAIN NUMBER 63, 0.63
TRAIN NUMBER 64, 0.64
TRAIN NUMBER 65, 0.65
TRAIN NUMBER 66, 0.66
TRAIN NUMBER 67, 0.67
TRAIN NUMBER 68, 0.68
TRAIN NUMBER 69, 0.69
TRAIN NUMBER 70, 0.7
TRAIN NUMBER 71, 0.71
TRAIN NUMBER 72, 0.72
TRAIN NUMBER 73, 0.73
TRAIN NUMBER 74, 0.74
TRAIN NUMBER 75, 0.75
TRAIN NUMBER 76, 0.76
TRAIN NUMBER 77, 0.77
TRAIN NUMBER 78, 0.78
TRAIN NUMBER 79, 0.79
TRAIN NUMBER 80, 0.8
TRAIN NUMBER 81, 0.81
TRAIN NUMBER 82, 0.82
TRAIN NUMBER 83, 0.83
TRAIN NUMBER 84, 0.84
TRAIN NUMBER 85, 0.85
TRAIN NUMBER 86, 0.86
TRAIN NUMBER 87, 0.87
TRAIN NUMBER 88, 0.88
TRAIN NUMBER 89, 0.89
TRAIN NUMBER 90, 0.9
TRAIN NUMBER 91, 0.91
TRAIN NUMBER 92, 0.92
TRAIN NUMBER 93, 0.93
TRAIN NUMBER 94, 0.94
TRAIN NUMBER 95, 0.95
TRAIN NUMBER 9

In [4]:
from sklearn.model_selection import train_test_split

training_data, test_data, train_target, test_target = train_test_split(combined_train_data, combined_late_amounts, train_size=0.8)
print('sample size = ', len(combined_train_data))
print('training_data size = ', len(training_data))
print('test_data size = ', len(test_data))

print(training_data)

sample size =  1200
training_data size =  960
test_data size =  240
      trainNumber  departureDate trainType.name trainType.trainCategory.name  \
5               6             11             IC                Long-distance   
495           837             11             IC                Long-distance   
994           416             11             IC                Long-distance   
340            55             11              S                Long-distance   
1178          177             11             IC                Long-distance   
...           ...            ...            ...                          ...   
657           162             11             IC                Long-distance   
89            265             11             IC                Long-distance   
195           416             11             IC                Long-distance   
284           185             11              S                Long-distance   
765           149             11             IC     